<a name='introduction'></a>
# Проект "Определение уровня английского для фильмов"

В данном проекте необходимо определить, какой уровень английского языка представлен в фильме. Уровень английского языка определяется в соответствии с уровнем Oxford CEFR. Определение уровня осуществляется на основе субтитров к фильму. В дополнение, можно добавлять фильмы и субтитры к ним из открытых источников, поскольку первоначально в дасете 241 фильм.

**План работы:**
1. [Предобработка данных](#data_preprocessing)
2. [Выбор метрики](#metric)
3. [Создание модели](#model)
4. [Анализ результатов](#analysis)
5. [Сохранение модели](#model_saving)

<a name='data_preprocessing'></a>
## 1. Предобработка данных
[Обратно к "Введению"](#introduction)

### 1. 1. Общий файл с фильмами

In [26]:
# Импортирование необходимых библиотек
import numpy as np
import pandas as pd
import PyPDF2
import pysrt
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

In [27]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /Users/kuuuzya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kuuuzya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kuuuzya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kuuuzya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/kuuuzya/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/kuuuzya/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
# Импорт файла
df_movies = pd.read_excel(f'Source/English_scores/movies_labels.xlsx')

df_movies.drop('id', axis=1, inplace=True)
df_movies.columns = ['movie', 'level']

print('Размер:', df_movies.shape)
df_movies.head()

Размер: (241, 2)


,movie,level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+


In [4]:
# Замена промежуточный уровень на нижний
df_movies.loc[df_movies['level'] == 'A2/A2+', 'level'] = 'A2'
df_movies.loc[df_movies['level'] == 'A2/A2+, B1', 'level'] = 'A2'
df_movies.loc[df_movies['level'] == 'B1, B2', 'level'] = 'B1'

df_movies['level_num'] = df_movies['level'].map({
    'A1': 0,
    'A2': 1,
    'B1': 2,
    'B2': 3,
    'C1': 4
})

df_movies.head(5)

,movie,level,level_num
0,10_Cloverfield_lane(2016),B1,2
1,10_things_I_hate_about_you(1999),B1,2
2,A_knights_tale(2001),B2,3
3,A_star_is_born(2018),B2,3
4,Aladdin(1992),A2,1


In [5]:
# Пропущенные значения
df_movies.isna().sum()

movie        0
level        0
level_num    0
dtype: int64

In [6]:
# Распределение количества фильмов по уровням 
df_movies['level'].value_counts()

level
B2    101
B1     63
C1     40
A2     37
Name: count, dtype: int64

### 1. 2. Файлы с субтитрами

In [10]:
# Вычленение то, что не относится к словам
HTML = r'<.*?>'
TAG = r'{.*?}'
COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]'
UPPER = r'[[A-Za-z ]+[\:\]]'
LETTERS = r'[^a-zA-Z\'.,!? ]'
SPACES = r'([ ])\1+'
DOTS = r'[\.]+'
SYMB = r"[^\w\d'\s]"


# Функция для загрузки и лемматизации субтитров
def saving_subs(film_loc):
    # Загрузка субтитров по кодированию
    film_subs = []
    encodings = ['', 'UTF-8-SIG', 'ISO-8859-1', 'utf-8', 'Windows-1252', 'ascii']
    encoding_number = 0

    # Проверка возможности правильности кодирования, иначе - использование другого
    while not film_subs:
        try:
            film_subs = pysrt.open(
                film_loc,
                encoding=encodings[encoding_number]
            )
        except UnicodeDecodeError:
            encoding_number += 1
    
    # Удаление лишнего и оставление только слов
    text = re.sub(HTML, ' ', film_subs[1:].text)
    text = re.sub(TAG, ' ', text)
    text = re.sub(COMMENTS, ' ', text)
    text = re.sub(UPPER, ' ', text)
    text = re.sub(LETTERS, ' ', text)
    text = re.sub(DOTS, r'.', text)
    text = re.sub(SPACES, r'\1', text)
    text = re.sub(SYMB, '', text)
    text = re.sub('www', '', text)
    text = text.lstrip()
    text = text.encode('ascii', 'ignore').decode()
    text = text.lower()

    # Лемматизация слов и добавление их в отдельный список
    film_words = []
    text_list = text.split()
    lemmatizer = WordNetLemmatizer()

    for i in range(len(text_list)):
        word = lemmatizer.lemmatize(text_list[i])
        # Проверка на наличие слова в списке и отсутствия в списке имен собственных
        if word not in film_words:
            film_words.append(word)

    # Объединение слов в одну строчку
    film_words = " ".join(film_words)
    return film_words

In [11]:
# Получение субтитров для каждого фильма, воспользовавшись функцией для обработки субтитров
for index, row in df_movies.iterrows():
    print(index)
    try:
        subs = saving_subs(f"Source/English_scores/subtitles_combined/{row['movie']}.srt")
        df_movies.loc[df_movies['movie'] == row['movie'], 'subs'] = subs
    except FileNotFoundError:
        pass

0


/var/folders/vd/cvcx7y592474yqnsftwldlpm0000gn/T/ipykernel_9329/3942574165.py:33: FutureWarning: Possible nested set at position 1
  text = re.sub(UPPER, ' ', text)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240


In [12]:
# Удаление фильмов, к которым не были найдены субтитры
df_movies = df_movies[(df_movies['subs'] != '') & (df_movies['subs'].notna())]

### 1. 3. Слова по уровню языка

Создадим список со всеми словами, а также отдельный словарь с ключами в виде уровней английского и значениями - индексами начала слов уровня - 1.

In [13]:
# Ссылки на слова (А1-C1)
a1_b2_link = 'Source/Oxford_CEFR_level/The_Oxford_3000_by_CEFR_level.pdf'
b2_c1_link = 'Source/Oxford_CEFR_level/The_Oxford_5000_by_CEFR_level.pdf'

In [14]:
# Функция для сохранения слов в список из PDF-файла 
def get_level_words(link):
    reader = PyPDF2.PdfReader(link)
    
    words = ''
    for n in range(len(reader.pages)):
        words += reader.pages[n].extract_text()
        
    words = words.split('\n')
    words = [words[n].split()[0] for n in range(len(words)) if n > 1]
    return words

In [15]:
# Сохранение слов по уровням в отдельные переменные
a1_b2_words = get_level_words(a1_b2_link)
b2_c1_words = get_level_words(b2_c1_link)

# Внесем некоторые корректировки
a1_b2_words[1] = 'a'
a1_b2_words.insert(2, 'an')
b2_c1_words.remove('3000')
b2_c1_words.remove('B2')

In [16]:
# Объединим слова
level_words = a1_b2_words
level_words.extend(b2_c1_words)
level_words[:10]

['A1',
 'a',
 'an',
 'about',
 'above',
 'across',
 'action',
 'activity',
 'actor',
 'actress']

In [17]:
# Создание словаря с ключами в виде уровня английского и значениями в виде индексов начала слов
# данного уровня минус единица
levels = ['A1', 'A2', 'B1', 'B2', 'C1']
levels_dict = {level: level_words.index(level) for level in levels}

print(levels_dict)

{'A1': 0, 'A2': 891, 'B1': 1752, 'B2': 2550, 'C1': 3958}


In [18]:
df_movies

,movie,level,level_num,subs
0,10_Cloverfield_lane(2016),B1,2,michelle please don't hang up just talk to me ...
1,10_things_I_hate_about_you(1999),B1,2,i'll be right with you so cameron here go nine...
2,A_knights_tale(2001),B2,3,should we help him he's due in the list two mi...
3,A_star_is_born(2018),B2,3,get to it black eye open wide it's time testif...
4,Aladdin(1992),A2,1,where the caravan camel roam it's flat and imm...
...,...,...,...,...
230,Suits.S03E06.720p.HDTV.x264-mSD,C1,4,i assume my deal with edward is dead a long yo...
231,Suits.S03E07.HDTV.x264-mSD,C1,4,it's going up on the wall tomorrow and this is...
232,Suits.S03E08.480p.HDTV.x264-mSD,C1,4,darby back me for managing partner i don't wan...
233,Suits.S03E09.480p.HDTV.x264-mSD,C1,4,i'm bonding with your father here speaking of ...


In [14]:
# Не улучшает качество моделей!


# # Определение количества слов по уровням для каждого фильма
# for index, row in df_movies.iterrows():
#     # Словарь с подсчетом слов по каждому уровню для отдельного фильма
#     film_level_words = {level: [] for level in levels}
    
#     # Добавление в словарь слов по их уровням
#     for word in row['subs'].split():
#         try:
#             word_index = level_words.index(word.lower())
#             if levels_dict['A1'] < word_index < levels_dict['A2']:
#                 if word not in film_level_words['A1']:
#                     film_level_words['A1'].append(word)
#             elif levels_dict['A2'] < word_index < levels_dict['B1']:
#                 if word not in film_level_words['A2']:
#                     film_level_words['A2'].append(word)
#             elif levels_dict['B1'] < word_index < levels_dict['B2']:
#                 if word not in film_level_words['B1']:
#                     film_level_words['B1'].append(word)
#             elif levels_dict['B2'] < word_index < levels_dict['C1']:
#                 if word not in film_level_words['B2']:
#                     film_level_words['B2'].append(word)
#             else:
#                 if word not in film_level_words['C1']:
#                     film_level_words['C1'].append(word)
#         except ValueError:
#             pass
    
#     # Подсчет количества слов по уровням
#     levels_words_count = {level: len(film_level_words[level]) for level in levels}
    
#     # Добавление количества слов по уровням в отдельные столбцы
#     total_words = sum(levels_words_count.values())
#     for level in levels:
#         df_movies.loc[df_movies['movie'] == row['movie'], level] = levels_words_count[level] / total_words


<a name='metric'></a>
## 2. Выбор метрики
[Обратно к "Введению"](#introduction)

Имеем дело с задачей мультиклассификации. В связи с этим необходимо использовать соответствующие метрики определения качества моделей. Назначение проекта - помочь учащимся, изучающих английский язык. В связи с этим лучше всего использовать F1-меру, поскольку она сочетает в себе и точность (precision), и полноту (recall). Однако, поскольку мы имеем дело с мультиклассификацией, то необходимо воспользоваться "Макро F1-мерой" (деомнстрирующей среднее значение F1-меры по классам) и "Микро F1-мерой" (глобальная средняя F1-меры, вычисляющая сумму TP, FN и FP).

Таким образом, для определения качества моделей будем использовать **f1_micro** и **f1_macro**.

<a name='model'></a>
## 3. Создание модели
[Обратно к "Введению"](#introduction)

### 3. 1. Разделение выборок

In [19]:
# Разделение датасета на тренировочную и тестовую выборки

X = df_movies['subs']
y = df_movies['level_num']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=df_movies['level_num']
)

In [37]:
# Функция для нахождения предсказаний и демонстрации метрик F1: микро и макро
vectorizer = TfidfVectorizer(stop_words='english')

def print_f1_score(model):
    pipe = Pipeline([
        ('vectorizer', vectorizer),
        ('model', model)
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    # Проверка модели на тестовой выборке
    print(f"{model}:")
    print("\tf1_micro: {:.4f}".format(f1_score(y_test, y_pred, average='micro')))
    print("\tf1_macro: {:.4f}".format(f1_score(y_test, y_pred, average='macro')))

### 3. 3. RandomForestClassifier

In [32]:
forest_model = RandomForestClassifier(random_state=42, n_estimators=350, max_depth=20)
print_f1_score(forest_model)

RandomForestClassifier(max_depth=20, n_estimators=350, random_state=42):
	f1_micro: 0.5106
	f1_macro: 0.3754


### 3. 4. LogisticRegression

In [33]:
logreg_model = LogisticRegression(random_state=42, max_iter=300, C=35)
print_f1_score(logreg_model)

LogisticRegression(C=35, max_iter=300, random_state=42):
	f1_micro: 0.5319
	f1_macro: 0.4204


### 3. 5. RidgeClassifier

In [34]:
ridge_model = RidgeClassifier(random_state=42, alpha=0.1)
print_f1_score(ridge_model)

RidgeClassifier(alpha=0.1, random_state=42):
	f1_micro: 0.5745
	f1_macro: 0.4843


### 3. 6. SGDClassifier

In [35]:
svc_model = SGDClassifier(random_state=42, loss='huber', alpha=0.0001, penalty='elasticnet', l1_ratio=0.01)
print_f1_score(svc_model)

SGDClassifier(l1_ratio=0.01, loss='huber', penalty='elasticnet',
              random_state=42):
	f1_micro: 0.5745
	f1_macro: 0.4921


### 3. 7. LinearSVC

In [36]:
svc_model = LinearSVC(random_state=42, C=3)
print_f1_score(svc_model)

LinearSVC(C=3, random_state=42):
	f1_micro: 0.5745
	f1_macro: 0.4843


<a name='analysis'></a>
## 4. Анализ результатов
[Обратно к "Введению"](#introduction)

Из 6 рассмотренных моделей наилучший результат по метрикам 'f1_micro' и 'f1_macro' продемонстрировали модели:
- RidgeClassifier(alpha=0.1, random_state=42):
	- f1_micro: 0.7857
	- f1_macro: 0.7684
- LinearSVC(C=3, random_state=42):
	- f1_micro: 0.7857
	- f1_macro: 0.7684
   
Поскольку необходимо использовать только одну модель, возьмем вторую: **LinearSVC(C=3, random_state=42)**, и сохраним полученную модель в отдельный файл для дальнейшего внедрения модели машинного обучения в веб-приложение по определению уровню английского языка, необходимого для понимания по меньшей мере 50-70% фильма.

<a name='model_saving'></a>
## 5. Сохранение модели
[Обратно к "Введению"](#introduction)

In [96]:
from pickle import dump

svc_pipe = Pipeline([
    ('vectorizer', vectorizer),
    ('model', svc_model)
])
    
svc_pipe.fit(X_train, y_train)

with open('./main.pcl', 'wb') as model_file:
    dump(svc_pipe, model_file)

In [100]:
y_pred = svc_pipe.predict(X_test)
predictions = pd.DataFrame(X_test)
predictions = a.merge(pd.Series(y_pred).rename('level_num'), right_index=True, left_index=True)
predictions.head(10)

,subs,level_num_x,level_num_y
48,pay careful attention to everything you see an...,4,4
50,saroo come on get up quickly hold it properly ...,3,3
5,itchy a few more degree to the left now tap no...,3,3
89,you've reached doug sorry i missed your call p...,2,2
7,and how it changed our valley forever there wa...,1,1
26,what give you that idea want to know the secre...,3,3
97,hi todd mahar eharmony how can i help you toda...,3,3
51,we keep moving on no messing around you mess t...,3,3
0,michelle please don't hang up just talk to me ...,3,3
84,there's a moment oforderly silence before foot...,1,1
